In [ ]:
import os
import sys
current_directory = os.getcwd()
if not os.path.exists(current_directory + '\\experimental_data'):
    os.makedirs(current_directory + '\\experimental_data')
    os.makedirs(current_directory + '\\experimental_data\\plots')
    os.makedirs(current_directory + '\\experimental_data\\results')
sys.path.append(current_directory.replace('experiments', ''))
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
plt.set_loglevel('WARNING')
from run import execute_simulation, create_experiment_snapshots, execute_ai_model, single_mtd_simulation, mtd_ai_simulation, multiple_mtd_simulation, specific_multiple_mtd_simulation
from mtdnetwork.mtd.completetopologyshuffle import CompleteTopologyShuffle
from mtdnetwork.mtd.ipshuffle import IPShuffle
from mtdnetwork.mtd.hosttopologyshuffle import HostTopologyShuffle
from mtdnetwork.mtd.portshuffle import PortShuffle
from mtdnetwork.mtd.osdiversity import OSDiversity
from mtdnetwork.mtd.servicediversity import ServiceDiversity
from mtdnetwork.mtd.usershuffle import UserShuffle
from mtdnetwork.mtd.osdiversityassignment import OSDiversityAssignment
import logging
import pandas as pd
import numpy as np
from math import pi
import matplotlib.pyplot as plt
from run_experiment import Experiment

logging.basicConfig(format='%(message)s', level=logging.INFO)

In [ ]:
# Learning Parameters
epsilon = 1.0  # exploration rate

# Simulator Settings
start_time = 0
finish_time = 15000
mtd_interval = [200]
network_size = [150]
total_nodes = 150
new_network = True

trial = 1500

mtd_strategies = [
    CompleteTopologyShuffle,
    # HostTopologyShuffle,
    IPShuffle,
    OSDiversity,
    # PortShuffle,
    # OSDiversityAssignment,
    ServiceDiversity,
    # UserShuffle
]

result_head_path = '/Users/williamho/Documents/GitHub/MTDSim'
static_degrade_factor = 2000



In [ ]:
# result = Experiment(epsilon, start_time, finish_time, mtd_interval, network_size,total_nodes, new_network, "all_features", trial,result_head_path)
# result.raw_result_stats_pipeline('nomtd',run_trial = True, stats_type = 'median')
# for scheme in result.schemes:
#     result.raw_result_stats_pipeline(scheme,run_trial = True, stats_type = 'median')

In [7]:
model = "all_features"
result = Experiment(model,epsilon, start_time, finish_time, mtd_interval, network_size,total_nodes, new_network, model, trial, result_head_path, mtd_strategies=mtd_strategies, static_degrade_factor=static_degrade_factor)
result.run_trials(model)


Trial_ 0
all_features


Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.


On Mac
Static_factor 2000
MTD Scheme [<class 'mtdnetwork.mtd.completetopologyshuffle.CompleteTopologyShuffle'>, <class 'mtdnetwork.mtd.ipshuffle.IPShuffle'>, <class 'mtdnetwork.mtd.osdiversity.OSDiversity'>, <class 'mtdnetwork.mtd.servicediversity.ServiceDiversity'>]
Action size(include zero which is no deployment) 5


Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.


mtd_ai
Trial_ 1
all_features
On Mac
Static_factor 2000
MTD Scheme [<class 'mtdnetwork.mtd.completetopologyshuffle.CompleteTopologyShuffle'>, <class 'mtdnetwork.mtd.ipshuffle.IPShuffle'>, <class 'mtdnetwork.mtd.osdiversity.OSDiversity'>, <class 'mtdnetwork.mtd.servicediversity.ServiceDiversity'>]
Action size(include zero which is no deployment) 5


...

mtd_ai
Trial_ 1497
all_features
On Mac
Static_factor 2000
MTD Scheme [<class 'mtdnetwork.mtd.completetopologyshuffle.CompleteTopologyShuffle'>, <class 'mtdnetwork.mtd.ipshuffle.IPShuffle'>, <class 'mtdnetwork.mtd.osdiversity.OSDiversity'>, <class 'mtdnetwork.mtd.servicediversity.ServiceDiversity'>]
Action size(include zero which is no deployment) 5


Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.


mtd_ai
Trial_ 1498
all_features
On Mac
Static_factor 2000
MTD Scheme [<class 'mtdnetwork.mtd.completetopologyshuffle.CompleteTopologyShuffle'>, <class 'mtdnetwork.mtd.ipshuffle.IPShuffle'>, <class 'mtdnetwork.mtd.osdiversity.OSDiversity'>, <class 'mtdnetwork.mtd.servicediversity.ServiceDiversity'>]
Action size(include zero which is no deployment) 5


Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.


mtd_ai
Trial_ 1499
all_features
On Mac
Static_factor 2000
MTD Scheme [<class 'mtdnetwork.mtd.completetopologyshuffle.CompleteTopologyShuffle'>, <class 'mtdnetwork.mtd.ipshuffle.IPShuffle'>, <class 'mtdnetwork.mtd.osdiversity.OSDiversity'>, <class 'mtdnetwork.mtd.servicediversity.ServiceDiversity'>]
Action size(include zero which is no deployment) 5
mtd_ai


In [ ]:

# static_features = ["host_compromise_ratio", "total_number_of_ports", "attack_path_exposure",  "overall_asr_avg", "roa", "shortest_path_variability", "risk"]
# time_features = ["mtd_freq", "overall_mttc_avg", "time_since_last_mtd"]

# features = static_features + time_features
# for model in features:

#     print(model)
#     result = Experiment(model, epsilon, start_time, finish_time, mtd_interval, network_size,total_nodes, new_network, model, trial, result_head_path)
#     result.run_trials(model)
    


In [ ]:
# # static_features = [ "overall_asr_avg", "attack_path_exposure"]
# static_features = []
# time_features = ["overall_mttc_avg"]
# metrics =  static_features + time_features


# for metric in metrics:
#     models = [
#         f"{metric}",
#         # f"{metric}_CompleteTopologyShuffle",
#         # f"{metric}_IPShuffle",
#         # f"{metric}_OSDiversity",
#         # f"{metric}_ServiceDiversity"
#     ]

#     mtd_strategies_dict = {
#         'CompleteTopologyShuffle': CompleteTopologyShuffle,
#         'IPShuffle': IPShuffle,
#         'OSDiversity': OSDiversity,
#         'ServiceDiversity': ServiceDiversity
#     }

#     for model in models:
#         # Extract strategy from model string
#         strategy_name = model.split('_')[-1]
        
#         # Find corresponding MTD strategy
#         mtd_strategy = mtd_strategies_dict.get(strategy_name, None)
        
#         if not mtd_strategy:
#             print(f"Strategy {strategy_name} not found in mtd_strategies_dict.")
#             continue
        
#         print(f"Running model: {model} with strategy: {strategy_name}")
        
#         # Create Experiment instance with the MTD strategy
#         result = Experiment(epsilon, start_time, finish_time, mtd_interval, network_size, total_nodes, new_network, model, trial, result_head_path, mtd_strategies=[mtd_strategy])
        
#         # Run trials
#         result.run_trials(model)